In [ ]:
# 1. Kütüphaneyi Kur
!pip install ultralytics -q

# 2. Drive'ı Bağla (Modeli Çekmek İçin)
from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

# 3. Modeli Colab'in içine kopyala (Daha hızlı çalışması için)
# Eğer klasör yapın farklıysa burayı kontrol et:
source_path = "/content/drive/MyDrive/YOLO_Models/baret_modeli_v1.pt"
destination_path = "/content/best.pt"

if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print("✅ Model başarıyla yüklendi! Hazırız.")
else:
    print("❌ HATA: Model Drive'da bulunamadı. Klasör yolunu kontrol et.")

Mounted at /content/drive
✅ Model başarıyla yüklendi! Hazırız.


In [ ]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
import os
import glob
from google.colab import files

# 1. Ortamdaki MP4 videolarını bul (sonuc videosu hariç)
video_files = [f for f in glob.glob("/content/*.mp4") if "sonuc" not in f]

if not video_files:
    print("❌ HATA: Hiç video bulunamadı!")
    print("Lütfen sol taraftaki dosya menüsüne .mp4 videonu sürükleyip bırak.")
else:
    # İlk bulunan videoyu al
    video_path = video_files[0]
    print(f"✅ Video bulundu: {video_path}")

    output_name = "sonuc_videosu.mp4"
    conf_threshold = 0.40

    # 2. Modeli Yükle
    print("Model yükleniyor...")
    # Modelin yerini de garantiye alalım (Drive'dan kopyalamıştık)
    if os.path.exists("/content/best.pt"):
        model_path = "/content/best.pt"
    else:
        # Drive bağlı değilse veya kopyalanmadıysa orjinal yolu dene
        model_path = "/content/runs/detect/hard_hat_safety_check/weights/best.pt"
        if not os.path.exists(model_path):
             print("❌ HATA: Model dosyası (best.pt) bulunamadı. Lütfen önce modeli yükle.")
             exit()

    model = YOLO(model_path)

    # 3. Videoyu İşle
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    if width == 0:
        print("❌ HATA: Video dosyası bozuk veya okunamıyor.")
    else:
        out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        print(f"İşlem başlıyor... ({width}x{height} @ {fps}fps)")

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            # Takip ve Çizim
            results = model.track(frame, persist=True, tracker="bytetrack.yaml",
                                  classes=[0, 1], conf=conf_threshold, verbose=False)

            safe_count = 0
            unsafe_count = 0

            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                cls_list = results[0].boxes.cls.int().cpu().tolist()

                for box, track_id, class_id in zip(boxes, track_ids, cls_list):
                    x, y, w, h = box
                    x1, y1 = int(x - w / 2), int(y - h / 2)
                    x2, y2 = int(x + w / 2), int(y + h / 2)

                    if class_id == 1: # Helmet
                        color = (0, 255, 0)
                        label = f"#{track_id} GUVENLI"
                        safe_count += 1
                    else: # Head
                        color = (0, 0, 255)
                        label = f"#{track_id} IHLAL!"
                        unsafe_count += 1

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Dashboard
            cv2.rectangle(frame, (10, 10), (280, 110), (0, 0, 0), -1)
            cv2.putText(frame, "IS GUVENLIGI SISTEMI", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(frame, f"Baretli : {safe_count}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(frame, f"IHLAL   : {unsafe_count}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            out.write(frame)

        cap.release()
        out.release()
        print(f"✅ İşlem Tamamlandı! İndiriliyor...")
        files.download(output_name)

✅ Video bulundu: /content/santiye_test_vid.mp4
Model yükleniyor...
İşlem başlıyor... (1280x720 @ 25fps)
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 115ms
Prepared 1 package in 21ms
Installed 1 package in 6ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

✅ İşlem Tamamlandı! İndiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import glob
import os
from google.colab import files

# --- 1. VİDEOYU OTOMATİK BUL ---
video_files = [f for f in glob.glob("/content/*.mp4") if "sonuc" not in f]
if not video_files:
    print("❌ HATA: Video bulunamadı! Lütfen videoyu yükle.")
    exit()

video_path = video_files[0]
output_name = "sonuc_bolge_analizi.mp4"
conf_threshold = 0.40

# --- 2. MODEL VE VİDEO AYARLARI ---
print(f"✅ Video işleniyor: {video_path}")
if os.path.exists("/content/best.pt"):
    model = YOLO("/content/best.pt")
else:
    model = YOLO("/content/runs/detect/hard_hat_safety_check/weights/best.pt")

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# --- 3. TEHLİKELİ BÖLGEYİ BELİRLE (SANAL ÇİT) ---
# Videonun ortasına bir dörtgen çiziyoruz.
# (Bunu gerçek projede kameranın açısına göre elle ayarlarsın)
# Koordinatlar: [Sol-Üst], [Sağ-Üst], [Sağ-Alt], [Sol-Alt]
zone_points = np.array([
    [int(width * 0.2), int(height * 0.2)], # Sol Üst (%20 içeride)
    [int(width * 0.8), int(height * 0.2)], # Sağ Üst
    [int(width * 0.8), int(height * 0.9)], # Sağ Alt
    [int(width * 0.2), int(height * 0.9)]  # Sol Alt
], np.int32)

zone_points = zone_points.reshape((-1, 1, 2))

print("İşlem başlıyor... Kahveni yudumla ☕")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # -- Bölgeyi Ekrana Çiz (Sarı Çizgiler) --
    cv2.polylines(frame, [zone_points], isClosed=True, color=(0, 255, 255), thickness=3)

    # Yarı saydam bir dolgu yapalım ki alan belli olsun
    overlay = frame.copy()
    cv2.fillPoly(overlay, [zone_points], (0, 255, 255))
    frame = cv2.addWeighted(overlay, 0.1, frame, 0.9, 0) # %10 saydam sarı

    # -- TAHMİN --
    results = model.track(frame, persist=True, tracker="bytetrack.yaml",
                          classes=[0, 1], conf=conf_threshold, verbose=False)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        cls_list = results[0].boxes.cls.int().cpu().tolist()

        for box, track_id, class_id in zip(boxes, track_ids, cls_list):
            x, y, w, h = box

            # Kutunun alt orta noktası (İşçinin ayak bastığı yer)
            # Bölgeye girip girmediğine buradan bakacağız.
            foot_x, foot_y = int(x), int(y + h / 2)

            # -- KRİTİK NOKTA: BÖLGE KONTROLÜ --
            # pointPolygonTest: Nokta çokgenin içinde mi?
            # Sonuç >= 0 ise İÇİNDE, < 0 ise DIŞINDA.
            is_inside = cv2.pointPolygonTest(zone_points, (foot_x, foot_y), False) >= 0

            if is_inside:
                # --- TEHLİKELİ BÖLGEDE ---
                if class_id == 1: # Helmet
                    color = (0, 255, 0) # Yeşil
                    status = "OK"
                else: # Head (Baretsiz) -> ALARM!
                    color = (0, 0, 255) # Kırmızı
                    status = "IHLAL!"

                    # Ekrana Kocaman ALARM Yazısı
                    cv2.putText(frame, "!!! ALARM !!!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
            else:
                # --- GÜVENLİ BÖLGEDE (DIŞARIDA) ---
                color = (192, 192, 192) # Gri
                status = "DISARIDA"

            # Çizimler
            x1, y1 = int(x - w / 2), int(y - h / 2)
            x2, y2 = int(x + w / 2), int(y + h / 2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.circle(frame, (foot_x, foot_y), 5, (255, 0, 0), -1) # Ayak noktasını göster

            label = f"#{track_id} {status}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Bitti! Video indiriliyor...")
files.download(output_name)

✅ Video işleniyor: /content/santiye_test_vid.mp4
İşlem başlıyor... Kahveni yudumla ☕
✅ Bitti! Video indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>